# Testing modular approach and master script for step integration

The objective of this testing notebook is to build a robust script capable of stacking reproductive event intervals where fetal mortality is likely to occur.

The code must be truly robust—capable of running and performing the task across the wide diversity of data in f4 & f5.

* Before anything, I need to get a testing dataset with all columns and multiple lactations from the original dataset.

I need multiple lactations, because I need the information of subsequent calving events to see if -283 days I do have an insemination.

What I will be looking for?

There are multiple criterias. I will need a closer look to define.
By now, just get samples w/repeated lactations.

In [1]:
import US25UW990JEPL_02 as JEPL
import copy
import csv
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
t=1

# Reading data

In [2]:
init = JEPL.Initializer(data_path='/blue/mateescu/agustinchasco/Projects/reprojersey/data',
                        code_path='/blue/mateescu/agustinchasco/Projects/reprojersey/core',
                        testing=t)

init.Directories()
init.GetFormatMap()

Initialization: 

Code ipykernel_launcher.py starts. 
Codes executed when testing is 1. 
Running program in /blue/mateescu/agustinchasco/Projects/reprojersey/core. 
Reading data in /blue/mateescu/agustinchasco/Projects/reprojersey/data. 
Saving data in /blue/mateescu/agustinchasco/Projects/reprojersey/data. 
Time profiling starts. Program executed on 05-16-2025 at 06:06 AM EST.

Path `/blue/mateescu/agustinchasco/Projects/reprojersey/data/ipykernel_launcher/test` already exists. 

Path `/blue/mateescu/agustinchasco/Projects/reprojersey/data/view` already exists. 

Loading map ipykernel_launcher_test_f4map.json in pathf4_map_path: 
 /blue/mateescu/agustinchasco/Projects/reprojersey/data/ipykernel_launcher/test/ipykernel_launcher_test_f4map.json 
 

Loading map ipykernel_launcher_test_f5map.json in pathf5_map_path: 
 /blue/mateescu/agustinchasco/Projects/reprojersey/data/ipykernel_launcher/test/ipykernel_launcher_test_f5map.json 
 

Creation of master map ipykernel_launcher_test_fmap.jso

In [3]:
print(f'JEPL020101 activated when testing is {init.t}. \n')
fn = ['US25UW990JEPL.01_impl_010601pdfu.txt', 'US25UW990JEPL.01_test_010601pdfuml.txt'][init.t]
fp = init.TargetPath(fn)

print(f'Loading file {fn} in path: \n {fp} \n \n')
dc = JEPL.DataContainer(init, pdfu = init.TXT_reader(target_path=fp))
dc.ShowDatasets()

JEPL020101 activated when testing is 1. 

Loading file US25UW990JEPL.01_test_010601pdfuml.txt in path: 
 /blue/mateescu/agustinchasco/Projects/reprojersey/data/US25UW990JEPL.02/test/US25UW990JEPL.01_test_010601pdfuml.txt 
 

init: <class 'US25UW990JEPL_02.Initializer'> (Not a DataFrame) 

pdfu: <class 'pandas.core.frame.DataFrame'> with shape (25001, 650) 



In [4]:
dc.SetKey(['pdfu'], [['f4X3', 'f4X4', 'f4X5', 'f4X35'], ['f5X3', 'f5X4', 'f5X5', 'f5X35']])

Setting key in merged dataset with 650 columns.


In [5]:
fdf = dc.pdfu
fdf.shape

(25001, 651)

# Scopes of my script.
Flatten each row, doin standardizations per row.

Code is not designed to cross information between rows.

Cross information between rows and hence do standardizations using the information of events below and above is going to be a subsequent step.

I think will be simpler if in this step I focus in the row, and then within rows.


# Notes:
- I see that sometimes testing data for a lactation is missing. By knowing the information in previous lactation, I can recover the missing lactation. 

This implies move accross rows, and I think that is safer if I do this in a subsequent step.

In [6]:
bb = fdf.iloc[:1000]

In [7]:
def check_YYYYMMDD(x):
    try:
        datetime.strptime(x, "%Y%m%d")
        return True
    except ValueError:
        return False

In [8]:
def ReturnStandardDate(date1, date2=None):
    if date2:
        b1 , b2 = check_YYYYMMDD(date1), check_YYYYMMDD(date2)
        b3 = b1 and b2 and date1 == date2
        if b3 or (b1 and not b2):
            return date1
        elif not b1 and b2:
            return date2
        elif not b1 and not b2:
            return 'n'*8
        else:
            print('Assertion error in `ReturnStandardDate`')
            assert False
    else:
        if check_YYYYMMDD(date1):
            return date1
        else:
            return 'n'*8
    

In [9]:
def DateDiff(date1, date2):
    b1 , b2 = check_YYYYMMDD(date1), check_YYYYMMDD(date2)
    b3 = b1 and b2
    if b3:
        d1 = datetime.strptime(date1, "%Y%m%d")
        d2 = datetime.strptime(date2, "%Y%m%d")
        d = (d2 - d1).days
        return f"{d:03d}" 
    else:
        return 'n'*3
        

In [10]:
def ReturnStandardCDAT(cdat, lact, diff, th):
    if lact == '00' or (diff != 'nnn' and int(diff) <= th):
        return '0'*8
    else:
        return ReturnStandardDate(date1=cdat, date2=None)

In [11]:
def ReturnStandardLACT(lactraw, diff, th):
    if lactraw == 'nn' and diff != 'nnn' and int(diff) <= th:
        return '00'
    else:
        return lactraw

In [12]:
def CheckTest(test):
    return any(v != '0'*len(v) and v != ' '*len(v) for v in test)

# Flattening
I see the temptation of jump between rows to recover the lactation number. However, I think that is not a good idea.
I am watching 200/5.4M of the data! What seems as a standard correction, often times is not.

This is why, in the flattening stage, I will keep it simple, and focus in `row` ignoring `row -1` and `row + 1`.

Result IS NOT PERFECT!! It need subsequent steps to remove new duplications, and recover infromation (for example, lactations).

In [13]:
# Criteria for identify Lact 0
th1 = int(365 + 365/2)

# Reproductive Set
rt = ['H', 'S', 'A', 'N', 'E', 'I', 'J', 'P', 'O', 'X', 'B', 'G']
sr = dc.GetColKey(colnames=['RES #'])
ks = dc.GetColKey(colnames=['Type of reproductive event'])
lr = len(sr)
ns = len(ks)
lsr = int(lr/ns)
th2 = lsr*(ns - 1)

# Testing Set
st = dc.GetColKey(colnames=['TDI #'])
ks = dc.GetColKey(colnames=['Days in milk this test day'])
lt = len(st)
ns = len(ks)
lst = int(lt/ns)
th3 = lst*(ns - 1)

In [41]:
# Flattening Reproductive events:
def ReproFlattening(row, threshold1=th2, threshold2=lsr, colkeysrepro=sr, validreprotypes=rt):
    base = 0
    repros = []
    while base <= threshold1:
        rtij = getattr(row, colkeysrepro[base])
        rdij = getattr(row, colkeysrepro[base+1])
        vtij, vdij = rtij in validreprotypes, check_YYYYMMDD(rdij)
        if vtij and vdij:
            step = 2
            ri = ['Repro', rdij, rtij]
            while step < threshold2:
                j = base + step
                ri.append(getattr(row, colkeysrepro[j]))
                step += 1
            repros.append(ri)
                
            base += threshold2
        else:
            base += threshold2
    return repros

In [15]:
# Flattening Testing events:
def TestFlattening(row, cdat, lact, threshold1=th3, threshold2=lst, colkeystest=st):
    tests = []
    vi1, vi2, vi3 = check_YYYYMMDD(cdat), lact != '00', lact != 'nn'
    if vi1 and vi2 and vi3:
        base = 0
        while base <= threshold1:
            step = 0
            ti = []
            while step < threshold2:
                j = base + step
                ti.append(getattr(row, colkeystest[j]))
                step += 1
            if CheckTest(ti):
                EVENT = 'Test'
                DATE = datetime.strptime(cdat, "%Y%m%d") + timedelta(days=int(ti[0]))
                DATE = DATE.strftime("%Y%m%d")
                ti = [EVENT] + [DATE] + ti
                tests.append(ti)
                base += threshold2
            else:
                base += threshold2
    return tests

In [16]:
# ID check:
def IDCheck(row, tid='ID'):
    if tid == 'ID':
        ID4 = row.f4X3 + row.f4X4 + row.f4X5
        ID5 = row.f5X3 + row.f5X4 + row.f5X5
    elif tid == 'SID':
        ID4 = row.f4X6 + row.f4X7 + row.f4X8
        ID5 = row.f5X6 + row.f5X7 + row.f5X8
    elif tid == 'DID':
        ID4 = row.f4X9 + row.f4X10 + row.f4X11
        ID5 = row.f5X9 + row.f5X10 + row.f5X11
    else:
        print('Function currently works for ID, SID or DID. Assertion error rised.')
        assert False
        
    n = len(ID4)
    d1 = 'n'*n
    d2 = ' '*n
    b1 = ID4 == d1 or ID4 == d2
    b2 = ID5 == d1 or ID5 == d2
    if b1 and b2:
        return d1
    elif not b1 and b2:
        return ID4
    else:
        return ID5

In [17]:
# Standardize a given value with defined output, crossing information between f4 & f5:
def StandardizeVal(row, keyf4, keyf5, validvals):
    val4 = getattr(row, keyf4)
    val5 = getattr(row, keyf5)
    vald = 'n'*len(val4)
    b1, b2 = val4 in validvals, val5 in validvals
    if not b1 and not b2:
        return vald
    elif b1 and not b2:
        return val4
    else:
        return val5

In [18]:
# Standardize a given values without defined output, crossing information between f4 & f5:
def StandardizeVal2(row, keyf4, keyf5):
    v1 = getattr(row, keyf4)
    v2 = getattr(row, keyf5)
    n = len(v1)
    d1 = 'n'*n
    d2 = ' '*n
    b1 = v1 == d1 or v1 == d2
    b2 = v2 == d1 or v2 == d2
    b3 = not b1 and not b2 and v1 == v2
    if b3 or (not b1 and b2):
        return v1
    elif b1 and not b2:
        return v2
    else:
        return d1

In [19]:
# Standardize one single value w/o crossing information between files:
def StandardizeVal3(row, key):
    v = getattr(row, key)
    n = len(v)
    d1 = 'n'*n
    d2 = ' '*n
    b1 = v == d1 or v == d2
    if not b1:
        return v
    else:
        return d1

In [20]:
# Standardize one single value w/o crossing information between files and asserting if value is in reported set.
def StandardizeVal4(row, key, validvals):
    v = getattr(row, key)
    if v in validvals:
        return v
    else:
        return 'n'*len(v)

In [21]:
# Standardize Date Cow Left Herd from YYMMDD to YYYYMMDD whenever possible.
def StandardizeDate(x):
    if x == 'n'*len(x):
        return 'n'*8
    else:
        yy = int(x[:2])
        if yy > 25:
            yyyy = '19' + str(yy)
        else:
            yyyy = '20' + str(yy)
        x = yyyy + x[2:]
        if check_YYYYMMDD(x=x):
            return x
        else:
            return 'n'*8

In [22]:
# Set of valid Miltiple Birth Codes:
mbc_set = [str(i) for i in range(1, 7)]

# Set of valid State codes
scs_set = ['11', '12', '13', '14', '15', '16', '21', '22', '23', '28', '31', '32', '33', '34', '35', '41', '42', '43', '45', '46', 
           '47', '48', '50', '51', '52', '54', '55', '56', '57', '58', '59', '59', '59', '59', '59', '61', '63', '64', '65', '66', 
           '71', '72', '73', '74', '75', '81', '82', '83', '84', '85', '86', '87', '88', '91', '92', '93', '94', '95', '96']

# Set of valid lactation type code
ltc_set = ['0', '1', '2', '4', '5', '6', '7', '8', '9', 'A', 'P', 'T']

# Set of valid lactation verification code
lvc_set = ['0', '1', '2', '3']

# Standardize first section:
def StandardizeMainSectiom(row, th1=th1, mbc_set=mbc_set, scs_set=scs_set, ltc_set=ltc_set, lvc_set=lvc_set):
    ID = IDCheck(row=row, tid='ID')
    SID = IDCheck(row=row, tid='SID')
    DID = IDCheck(row=row, tid='DID')
    BDAT = ReturnStandardDate(date1=row.f4X15, date2=row.f5X15)
    CDAT = ReturnStandardCDAT(cdat=ReturnStandardDate(date1=row.f4X35, date2=row.f5X35), 
                              lact=row.f4X43, diff=DateDiff(date1=BDAT, date2=ReturnStandardDate(date1=row.f5X38)), 
                              th=th1)
    LACT = ReturnStandardLACT(lactraw=row.f4X43, diff=DateDiff(date1=BDAT, date2=ReturnStandardDate(date1=row.f5X38)), 
                              th=th1)
    KEY = ID + BDAT + CDAT
    MBC = StandardizeVal(row=row, keyf4='f4X21', keyf5='f5X21', validvals=mbc_set)
    SC = StandardizeVal(row=row, keyf4='f4X28', keyf5='f5X28', validvals=scs_set)
    CC = StandardizeVal2(row=row, keyf4='f4X29', keyf5='f5X29')
    HN = StandardizeVal2(row=row, keyf4='f4X30', keyf5='f5X30')
    CCN = StandardizeVal2(row=row, keyf4='f4X31', keyf5='f5X31')
    DCLH = StandardizeDate(x=StandardizeVal3(row=row, key='f4X32'))
    LTC = StandardizeVal4(row=row, key='f4X33', validvals=ltc_set)
    LVC = StandardizeVal(row=row, keyf4='f4X34', keyf5='f5X34', validvals=lvc_set)
    return [KEY, ID, BDAT, CDAT, LACT, SID, DID, MBC, SC, CC, HN, CCN, DCLH, LTC, LVC]

In [30]:
# Set of valid Primary Termination Codes (f4X44)
ptc_set = ['0', '8', '2', '1', '3', '4', '7', '9', 'A', 'B', '5', '6']

# Set of valid Weight Reported Code (f4X47)
wrc_set = [str(i) for i in range(10)]

# Set of valid Lactation Initiation Code (f4X48)
lic_set = ['0', '1', '2', '8', '9']

# Set of Milking/dry status (0=not used, 1=milking, 2=dry) (f4X49)
mds_set = ['0', '1', '2']

# Set of valid True Protein Codes (f4X54)
tpc_set = ['0', '1']

# Set of valid Testing Method Codes (f4X55)
tmc_set = ['0', '1', '2', '3']

# Set of valid Quality Certification status (f4X56)
qc_set = ['1', '2', '3', '4']

# Set of valid Pregnancy Confirmation Codes (f4X70)
pcc_set = ['0', '1', '2', '3']

# Set of valid Number of Progeny Born at this Calving (f4X71)
npb_set = ['0', '1', '2', '3', '4']

# Set of valid Secondary Termination Codes (f4X72)
stc_set = ptc_set

# Excluded fields:
excl = ['f4X43'] # Lactation Number

# Date fields
crf = ['f4X45']

# Fields that map potentiually important data that is uniquely found in f4
unqf4_set1 = ['f4X' + str(i) for i in range(36, 73) if 'f4X' + str(i) not in excl]

# Fields that map potentiually important data that is uniquely found in f4 with a defined set of codes:
trgf4_dict = {'f4X44': ptc_set,
              'f4X47': wrc_set,
              'f4X48': lic_set,
              'f4X49': mds_set,
              'f4X54': tpc_set,
              'f4X55': tmc_set,
              'f4X56': qc_set,
              'f4X70': pcc_set,
              'f4X71': npb_set,
              'f4X72': stc_set}

unqf4_set2 = list(trgf4_dict.keys())


# Standardize disjoint section of f4 w/ milk production:
def Standardizef4Disjoint(row, unqf4_set1=unqf4_set1, unqf4_set2=unqf4_set2, crf=crf):
    f4disj = []
    for field in unqf4_set1:
        if field == crf:
            f4disj.append(ReturnStandardDate(date1=getattr(row, field)))
        elif field not in unqf4_set2:
            f4disj.append(StandardizeVal3(row=row, key=field))
        else:
            f4disj.append(StandardizeVal4(row=row, key=field, validvals=trgf4_dict[field]))
    return f4disj
unqf4_set1

In [39]:
# Column name for main section (will be usefull when writting lines):
cs1 = ['KEY', 'ID', 'BDAT', 'CDAT', 'LACT', 'SID', 'DID', 'MBC', 'SC', 'CC', 'HN', 'CCN', 'DCLH', 'LTC', 'LVC']

# Column names for production section:
cs2 = ['DIM1', 'DPTL', 'NTDL', 'DM3TD', 'ALMY', 'ALFY', 'ALPY', 'PTC', 'DOBRIC', 'BWSL', 'WRC', 'LIC', 
       'MDS', 'TTPC', 'TLMSCS', 'LASCS', 'NTSCS', 'TPC', 'TMC', 'QCS', 'LSMY', 'LSFY', 'LSPY', 'MYD', 
       'FYD', 'PYD', 'NLIEGC', 'BNA', 'NAABBC', 'NST', 'NTCA', 'DCRY', 'DCRSCS', 'PCC', 'NPBTC', 'STC']

# Column names for test segments:
cts = ['EVENT', 'DATE', 'DIM2', 'SUPC', 'LTDSC', 'MF', 'NMMW', 'NMSTA', 'MRD', 'PSM', 'AMY', 'AFP', 'APP', 'ASCS']

# Column names for reproductive segments:
crs = ['EVENT', 'DATE', 'TRE', ]



In [23]:
## Potentially delete:
# Animal Herd Code. Set in f4 & f5

#, 'LactationTypeCode', 'LactationVerificationCode'
f4k = ['f4X31', 'f4X32']
f5k = ['f5X31', 'f5X32']
fkn = ['X31', 'X32']

fg1 = 'n'*8

In [42]:
for row in bb.itertuples(index=False):
    # Shared and main section:
    shared = StandardizeMainSectiom(row=row)
    
    # Productive Section from f4
    milkpr = Standardizef4Disjoint(row=row)
    
    # Flattening Testing events
    tests = TestFlattening(row, cdat=shared[3], lact=shared[4])
    
    # Flattening Reproductive events
    repros = ReproFlattening(row=row)
    
    print({'shared': shared, 
           'milkpr': milkpr, 
           'tests': tests, 
           'repros': repros})
    print()
    
    
    

{'shared': ['JE8400032103897482019091600000000', 'JE840003210389748', '20190916', '00000000', '00', 'JE840003132350040', 'JEUSA000067914421', '1', '93', '50', '1216', '17587', '20200317', 'T', '0'], 'milkpr': ['000', '000', '00', '000', '0000', '0000', '0000', '6', '00000000', '000', '0', '0', '0', '00', '0', '000', '000', '0', '0', 'n', '00000', '0000', '0000', '+00000', '+0000', '+0000', '0000', 'JAN    ', '0151', '00', '00', '000', '000', '0', '0', 'n'], 'tests': [], 'repros': []}

{'shared': ['JE8400032103897492019091600000000', 'JE840003210389749', '20190916', '00000000', '00', 'JEUSA000067742125', 'JEUSA000072721314', '1', '93', '50', '1216', '17588', 'nnnnnnnn', 'n', '0'], 'milkpr': ['nnn', 'nnn', 'nn', 'nnn', 'nnnn', 'nnnn', 'nnnn', 'n', 'nnnnnnnn', 'nnn', 'n', 'n', 'n', 'nn', 'n', 'nnn', 'nnn', 'n', 'n', 'n', 'nnnnn', 'nnnn', 'nnnn', 'nnnnnn', 'nnnnn', 'nnnnn', 'nnnn', 'nnnnnnn', 'nnnn', 'nn', 'nn', 'nnn', 'nnn', 'n', 'n', 'n'], 'tests': [], 'repros': [['Repro', '20201013', 'G

In [41]:
check = bb[bb['f4X75'] != 'nnn']
ktr1 = dc.GetColKey(colnames=['TDI #1 '])
ktr2 = dc.GetColKey(colnames=['TDI #2 '])
check[['Key'] + ktr1]

,Key,f4X75,f4X76,f4X77,f4X78,f4X79,f4X80,f4X81,f4X82,f4X83,f4X84,f4X85,f4X86
0,JE840003210389748,,,,,,,,,,,,
3,JE840003210389750,,,,,,,,,,,,
5,JE84000321038975120210905,009,1,3,2,0,0,00,102,0000,00,00,
15,JE84000321038975620210909,005,1,1,2,1,1,01,102,0319,54,34,54
21,JE84000321038975920210927,015,1,1,2,1,1,01,101,0379,35,31,38
25,JE84000321038976120210712,029,1,1,2,1,1,01,100,0456,56,35,35
33,JE84000321038976520210612,031,1,1,2,1,1,01,102,0497,47,32,67
34,JE840003210389766,,,,,,,,,,,,
40,JE84000321038976920210709,000,0,0,0,0,0,00,000,0000,00,00,00
42,JE84000321038977020210820,025,1,1,2,1,1,01,102,0479,48,34,06


# Stop!

# Fundamental improvement.
Calving date often times recorded in one file but not recorded in other.
Whenever this happens, `key` is corrupted. Then, 2 different Keys may reference the same animal in two different rows. One row has testing data ande not repro data, while other row has repro data but no testing data.
The problem..., is that ID + BDAT is not enough to identify the lactation we are talking bout.

In [13]:
rt = ['A', 'N', 'E', 'I', 'J', 'B', 'G']

In [11]:
# Type of reproductive event. It is A, O, P H, etc.
kt = dc.GetColKey(colnames=['Type of reproductive event'])
# I also have the column with the dates:
kd = dc.GetColKey(colnames=['Date of reproductive event'])

I will need to make sure dates are in fact YYYYMMDD.

In [14]:
stp = 20
cnt = 0
n = len(kt)
for r in range(fdf.shape[0]):
    print('-------- START---------')
    # ??? sub 
    ki = 0
    kf = ki+1
    li = 1
    lf = li+1
    print('cnt = ', cnt)
    while kf <= n-1:
        ti = fdf.at[r, kt[ki]]
        tf = fdf.at[r, kt[kf]]
        di = fdf.at[r, kd[ki]]
        df = fdf.at[r, kd[kf]]
        if ti not in rt:
            ki = kf
            kf = ki + 1
        elif ti in rt and not check_YYYYMMDD(x=di):
            li = lf
            lf = li + 1
            ki = kf
            kf = ki + 1
        elif tf not in rt:
            kf += 1
        elif tf in rt and not check_YYYYMMDD(x=df):
            lf += 1
            kf += 1
        elif ti in rt and tf in rt and check_YYYYMMDD(x=di) and check_YYYYMMDD(x=df):
            did = datetime.strptime(di, "%Y%m%d")
            dfd = datetime.strptime(df, "%Y%m%d")
            d = (dfd - did).days
            l = str(li) + ti + str(lf) + tf
            print(fdf.at[r, 'Key'], ': ', l, '->', d)
            li = lf
            lf = li + 1
            ki = kf
            kf = ki + 1
        else:
            print('why?')
    if cnt == stp:
        break
    cnt += 1
    print('-------- END---------\n\n')

-------- START---------
cnt =  0
-------- END---------


-------- START---------
cnt =  1
-------- END---------


-------- START---------
cnt =  2
-------- END---------


-------- START---------
cnt =  3
-------- END---------


-------- START---------
cnt =  4
JE84000321038975100000000 :  1G2G -> 22
JE84000321038975100000000 :  2G3G -> 23
-------- END---------


-------- START---------
cnt =  5
-------- END---------


-------- START---------
cnt =  6
JE84000321038975200000000 :  1G2G -> 20
JE84000321038975200000000 :  2G3G -> 19
-------- END---------


-------- START---------
cnt =  7
-------- END---------


-------- START---------
cnt =  8
JE84000321038975300000000 :  1G2G -> 21
-------- END---------


-------- START---------
cnt =  9
JE84000321038975320210807 :  1G2G -> 22
JE84000321038975320210807 :  2G3A -> 23
JE84000321038975320210807 :  3A4A -> 23
-------- END---------


-------- START---------
cnt =  10
JE84000321038975400000000 :  1G2A -> 44
-------- END---------


-------- STA

I need to make sure my code is working as expected.
Check with raw data.

In [15]:
fdf[kt].head(20)

,f5X37,f5X52,f5X67,f5X82,f5X97,f5X112,f5X127,f5X142,f5X157,f5X172,f5X187,f5X202,f5X217,f5X232,f5X247,f5X262,f5X277,f5X292,f5X307,f5X322
0,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n
1,G,P,,,,,,,,,,,,,,,,,,
2,G,P,,,,,,,,,,,,,,,,,,
3,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n
4,G,G,G,P,,,,,,,,,,,,,,,,
5,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n,n
6,G,G,H,G,P,,,,,,,,,,,,,,,
7,H,G,P,,,,,,,,,,,,,,,,,
8,G,G,P,,,,,,,,,,,,,,,,,
9,G,G,A,A,P,,,,,,,,,,,,,,,


# Werite result - START SIMPLE PLEASE!

In [16]:
n = len(kt)
mo = {'Key': 'Key', 
      'f4X43': 'Lactation', 
      'f4X45': 'DOBRIC'}
oc = [mo[k] for k in mo]
oo = list(mo)
ec = ['Repro_Label', 'Repro_Date_i', 'Repro_Date_f', 'Interval_Days']
c = oc + ec
fn = dc.init.SavePath(extension='flat.txt', dirct='data', mode=0)

with open(fn, mode="w") as f:
    f.write('\t'.join(c) + '\n')
    for _, row in fdf.iterrows():
        vs = [str(row[j]) for j in oo]
        ki, li = 0, 1
        kf, lf = 1, 2
        while kf < n-1:
            ti, tf = row[kt[ki]], row[kt[kf]]
            di, df = row[kd[ki]], row[kd[kf]]
            vdi, vdf = check_YYYYMMDD(di), check_YYYYMMDD(df)
            
            if ti not in rt:
                ki, kf = kf, kf + 1
            elif ti in rt and not vdi:
                li, lf = lf, lf + 1
                ki, kf = kf, kf + 1
            elif tf not in rt:
                kf += 1
            elif tf in rt and not vdf:
                lf += 1
                kf += 1
            elif vdi and vdf:
                did = datetime.strptime(di, "%Y%m%d")
                dfd = datetime.strptime(df, "%Y%m%d")
                d = (dfd - did).days
                l = str(li) + ti + str(lf) + tf
                ad = [l, di, df, str(d)]
                f.write('\t'.join(vs + ad) + '\n')
                li, lf = lf, lf + 1
                ki, kf = kf, kf + 1
            else:
                print("why? Please check!")
                print(row[kt])
                print(row[kd])
                break

In [17]:
check = fdf[fdf['Key'] == 'JE84000321038975200000000']
check

,f4X1,f4X2,f4X3,f4X4,f4X5,f4X6,f4X7,f4X8,f4X9,f4X10,...,f5X328,f5X329,f5X330,f5X331,f5X332,f5X333,f5X334,f5X335,f5X336,Key
6,n,n,nn,nnn,nnnnnnnnnnnn,nn,nnn,nnnnnnnnnnnn,nn,nnn,...,,,,,,,,,,JE84000321038975200000000


In [18]:
with open(fn[:-4] + '_check.html', 'w', encoding='utf-8') as f:
    f.write(check.to_html(index=False, border=1))

# Add the following:
- ID
- CDAT

In [19]:
n = len(kt)
mo = {'Key': 'Key', 
      'f4X43': 'Lactation', 
      'f4X45': 'DOBRIC'}
oc = [mo[k] for k in mo]
oo = list(mo)
ec = ['ID', 'CDAT', 
      'Repro_Label', 'Repro_Date_i', 'Repro_Date_f', 'Interval_Days']
c = oc + ec
fn = dc.init.SavePath(extension='flat.txt', dirct='data', mode=0)

with open(fn, mode="w") as f:
    f.write('\t'.join(c) + '\n')
    for _, row in fdf.iterrows():
        vs = [str(row[j]) for j in oo]
        ID = row['Key'][:-8]
        CDAT = row['Key'][-8:]
        vs = vs + [ID, CDAT]
        ki, li = 0, 1
        kf, lf = 1, 2
        while kf < n-1:
            ti, tf = row[kt[ki]], row[kt[kf]]
            di, df = row[kd[ki]], row[kd[kf]]
            vdi, vdf = check_YYYYMMDD(di), check_YYYYMMDD(df)
            
            if ti not in rt:
                ki, kf = kf, kf + 1
            elif ti in rt and not vdi:
                li, lf = lf, lf + 1
                ki, kf = kf, kf + 1
            elif tf not in rt:
                kf += 1
            elif tf in rt and not vdf:
                lf += 1
                kf += 1
            elif vdi and vdf:
                did = datetime.strptime(di, "%Y%m%d")
                dfd = datetime.strptime(df, "%Y%m%d")
                d = (dfd - did).days
                l = str(li) + ti + str(lf) + tf
                ad = [l, di, df, str(d)]
                f.write('\t'.join(vs + ad) + '\n')
                li, lf = lf, lf + 1
                ki, kf = kf, kf + 1
            else:
                print("why? Please check!")
                print(row[kt])
                print(row[kd])
                break

# You need to add SID & DID  
- Focus on SID  

You will need to check this information in both f4 & f5 files.  
- If consistent, add DID  
- If not consistent, see why such inconsistency occurs:  
  - a) Something super basic and obvious. Then you solve it.  
  - b) Something that actually needs an unknown assumption: add nothing.  

In [21]:
def ID_verifier(row, id_type='ID', fg='n'*17):
    if id_type == 'ID':
        k4 = ['f4X3', 'f4X4', 'f4X5']
        k5 = ['f5X3', 'f5X4', 'f5X5']
    elif id_type == 'SID':
        k4 = ['f4X6', 'f4X7', 'f4X8']
        k5 = ['f5X6', 'f5X7', 'f5X8']
    elif id_type == 'DID':
        k4 = ['f4X9', 'f4X10', 'f4X11']
        k5 = ['f5X9', 'f5X10', 'f5X11']
    else:
        print('Function ID_verifier designed to capture `id_type` as `ID`, `SID` or `DID`. Please...., check.')
        assert False
        
    f4ID = row[k4[0]] + row[k4[1]] + row[k4[2]]
    f5ID = row[k5[0]] + row[k5[1]] + row[k5[2]]
    
    b4 = f4ID == fg 
    b5 = f5ID == fg
    
    if b4 and b5:
        return fg
    elif b4 and not b5:
        return f5ID
    elif not b4 and b5:
        return f4ID
    elif not b4 and not b5:
        if f4ID == f5ID:
            return f4ID
        else:
            return fg
    else:
        print('Logical bug in function `ID_verifier`. Check function..., eat the bug Pumbaa!')
        assert False

In [22]:
c4 = [c for c in fdf.columns if 'f4' in c]
c5 = [c for c in fdf.columns if 'f5' in c]

for _, row in fdf.iterrows():
        f4bd = row['f4X15']
        f5bd = row['f5X15']
        if f4bd != 'n'*8 and f5bd != 'n'*8 and f4bd != f5bd:
            print('\n------------\nKey = ', row['Key'], '\nf4BD = ', f4bd, '\nf5BD = ', f5bd, '\n------------\n')
            r4 = row[['Key'] + c4]
            r5 = row[['Key'] + c5]
            # print([r4, r5][1])


------------
Key =  JE84000321184720320211023 
f4BD =  20191111 
f5BD =  20190507 
------------


------------
Key =  JE84000321184723220211021 
f4BD =  20191116 
f5BD =  20190505 
------------


------------
Key =  JE84000321184731820211027 
f4BD =  20191207 
f5BD =  20190511 
------------


------------
Key =  JE84000321184732320211016 
f4BD =  20191209 
f5BD =  20190430 
------------


------------
Key =  JE84000321184733120211025 
f4BD =  20191212 
f5BD =  20190509 
------------


------------
Key =  JE84000321184737220211021 
f4BD =  20191221 
f5BD =  20190505 
------------


------------
Key =  JE84000321184738920211016 
f4BD =  20191223 
f5BD =  20190430 
------------


------------
Key =  JE84000321184739620211016 
f4BD =  20191225 
f5BD =  20190430 
------------


------------
Key =  JE84000321184741520211019 
f4BD =  20191228 
f5BD =  20190503 
------------


------------
Key =  JE84000321184741720211029 
f4BD =  20191228 
f5BD =  20190513 
------------


------------
Key = 

Potential logic bug: you are not writting anything when f4 has 'n'. Potentially, missing calving date in f4 or f5 for same ID results in different Key.

Due sorting, such keys must be one next to the other.

By writting repro event with 'n', you will be able to identify when 2 different keys are in fact from the same animal.

In [28]:
fdf.shape

(25001, 651)

In [40]:
n = len(kt)
mo = {'Key': 'Key', 
      'f4X43': 'Lactation', 
      'f4X45': 'DOBRIC'}
oc = [mo[k] for k in mo]
oo = list(mo)
ec = ['ID', 'CDAT', 'BDAT', 'SID', 'DID',
      'Repro_Label', 'Repro_Date_i', 'Repro_Date_f', 'Interval_Days']
c = oc + ec

fn = dc.init.SavePath(extension='flat.txt', dirct='data', mode=0)
fg17 = 'n'*17
fg8 = 'n'*8

c4 = [c for c in fdf.columns if 'f4' in c]
c5 = [c for c in fdf.columns if 'f5' in c]

with open(fn, mode="w") as f:
    f.write('\t'.join(c) + '\n')
    for _, row in fdf.iterrows():
        bd4 = row['f4X15']
        bd5 = row['f5X15']
        k0 = row['Key']
        rs = []
        
        b1 = bd4 == fg8
        b2 = bd5 == fg8
        b3 = bd4 == bd5
        
        if not b1 and not b2 and not b3:
            r4 = row.copy()
            r4['Key'] = k0 + bd4
            r4.update({c: 'n'*len(row[c]) for c in c5})
            rs.append(r4)
            r5 = row.copy()
            r5['Key'] = k0 + bd5
            r5.update({c: 'n'*len(row[c]) for c in c4})
            rs.append(r5)
            
        elif b1 and not b2:
            r = row.copy()
            r['Key'] = k0 + bd5
            rs.append(r)
            
        elif not b1 and b2:
            r = row.copy()
            r['Key'] = k0 + bd4
            rs.append(r)
            
        elif b3:
            r = row.copy()
            r['Key'] = k0 + bd4
            rs.append(r)
            
        else:
            print('\n----------\nbd4 = ', bd4, '\nbd5 = ', bd5, '\n----------\n')
            print('Please check assertion error! Buggy logic!')
            assert False
            
        for r in rs:
            vs = [str(r[j]) for j in oo]
            ID = r['Key'][:17]
            CDAT = r['Key'][17:25]
            BDAT = r['Key'][25:]
            SID = ID_verifier(row=r, id_type='SID', fg=fg17)
            DID = ID_verifier(row=r, id_type='DID', fg=fg17)
            vs = vs + [ID, CDAT, BDAT, SID, DID]
            ki, li = 0, 1
            kf, lf = 1, 2
            while kf <= n-1:
                ti, tf = r[kt[ki]], r[kt[kf]]
                di, df = r[kd[ki]], r[kd[kf]]
                vdi, vdf = check_YYYYMMDD(di), check_YYYYMMDD(df)
                
                if ti == 'n':
                    l = 'nnnn'
                    ad = [l, 'nnnnnnnn', 'nnnnnnnn', 'n']
                    f.write('\t'.join(vs + ad) + '\n')
                    break
                
                if ti not in rt:
                    ki, kf = kf, kf + 1
                elif ti in rt and not vdi:
                    li, lf = lf, lf + 1
                    ki, kf = kf, kf + 1
                elif tf not in rt:
                    kf += 1
                elif tf in rt and not vdf:
                    lf += 1
                    kf += 1
                elif vdi and vdf:
                    did = datetime.strptime(di, "%Y%m%d")
                    dfd = datetime.strptime(df, "%Y%m%d")
                    d = (dfd - did).days
                    l = str(li) + ti + str(lf) + tf
                    ad = [l, di, df, str(d)]
                    f.write('\t'.join(vs + ad) + '\n')
                    li, lf = lf, lf + 1
                    ki, kf = kf, kf + 1
                else:
                    print("why? Please check!")
                    print(r[kt])
                    print(r[kd])
                    break

# What next?
- VWP: add interval.
- Add last interval. For example, 1A2P must be recorded. ==> Hold on. This is a challenging step. Things must be standardized.

In [29]:
n = len(kt)
mo = {'Key': 'Key', 
      'f4X43': 'Lactation', 
      'f4X45': 'DOBRIC'}
oc = [mo[k] for k in mo]
oo = list(mo)
ec = ['ID', 'CDAT', 'BDAT', 'SID', 'DID',
      'VWP', 'Repro_Label', 'Repro_Date_i', 'Repro_Date_f', 'Interval_Days']
c = oc + ec

fn = dc.init.SavePath(extension='flat.txt', dirct='data', mode=0)
fg17 = 'n'*17
fg8 = 'n'*8

c4 = [c for c in fdf.columns if 'f4' in c]
c5 = [c for c in fdf.columns if 'f5' in c]

with open(fn, mode="w") as f:
    f.write('\t'.join(c) + '\n')
    for _, row in fdf.iterrows():
        bd4 = row['f4X15']
        bd5 = row['f5X15']
        k0 = row['Key']
        rs = []
        
        b1 = bd4 == fg8
        b2 = bd5 == fg8
        b3 = bd4 == bd5
        
        if not b1 and not b2 and not b3:
            r4 = row.copy()
            r4['Key'] = k0 + bd4
            r4.update({c: 'n'*len(row[c]) for c in c5})
            rs.append(r4)
            r5 = row.copy()
            r5['Key'] = k0 + bd5
            r5.update({c: 'n'*len(row[c]) for c in c4})
            rs.append(r5)
            
        elif b1 and not b2:
            r = row.copy()
            r['Key'] = k0 + bd5
            rs.append(r)
            
        elif not b1 and b2:
            r = row.copy()
            r['Key'] = k0 + bd4
            rs.append(r)
            
        elif b3:
            r = row.copy()
            r['Key'] = k0 + bd4
            rs.append(r)
            
        else:
            print('\n----------\nbd4 = ', bd4, '\nbd5 = ', bd5, '\n----------\n')
            print('Please check assertion error! Buggy logic!')
            assert False
            
        for r in rs:
            vs = [str(r[j]) for j in oo]
            ID = r['Key'][:17]
            CDAT = r['Key'][17:25]
            BDAT = r['Key'][25:]
            SID = ID_verifier(row=r, id_type='SID', fg=fg17)
            DID = ID_verifier(row=r, id_type='DID', fg=fg17)
            vs += [ID, CDAT, BDAT, SID, DID]
            ki, li = 0, 1
            kf, lf = 1, 2
            vwp = 'nnn'
            while kf <= n-1:
                ti, tf = r[kt[ki]], r[kt[kf]]
                di, df = r[kd[ki]], r[kd[kf]]
                vdc, vdi, vdf = check_YYYYMMDD(CDAT), check_YYYYMMDD(di), check_YYYYMMDD(df)
                
                if ti == 'n':
                    l = 'nnnn'
                    ad = [vwp, l, 'nnnnnnnn', 'nnnnnnnn', 'n']
                    f.write('\t'.join(vs + ad) + '\n')
                    break
                
                if ti not in rt:
                    ki, kf = kf, kf + 1
                elif ti in rt and not vdi:
                    li, lf = lf, lf + 1
                    ki, kf = kf, kf + 1
                elif tf not in rt:
                    kf += 1
                elif tf in rt and not vdf:
                    lf += 1
                    kf += 1
                elif vdi and vdf:
                    did = datetime.strptime(di, "%Y%m%d")
                    dfd = datetime.strptime(df, "%Y%m%d")
                    d = (dfd - did).days
                    if vdc and vwp == 'nnn':
                        dcd = datetime.strptime(CDAT, "%Y%m%d")
                        vwp = (did - dcd).days
                    l = str(li) + ti + str(lf) + tf
                    ad = [str(vwp), l, di, df, str(d)]
                    f.write('\t'.join(vs + ad) + '\n')
                    li, lf = lf, lf + 1
                    ki, kf = kf, kf + 1
                else:
                    print("Why? Please check!")
                    print(r[kt])
                    print(r[kd])
                    assert False

# CRF
I do have 25000 rows..., after flattening..., I do have 255000 rows.... That does not makes sense! I expect to have much more!

Assume 50% of 25K has Repro data. ==> 12.5 base rows w/o Repro data + 12.5 * N Repr Ev.

Among reports with repro events..., I assume to have on average at least 2 valid repro events per lactation. 

Then: 12.5*2 = 25

Then, I expect: 12.5 + 25 =37.5 37.5K records.

I think..., some rows do not have any valid repro event.., and the `while` loop is breaking before writting any line. You need to be able to write at least one line pre iterated row!

You are not supposed to filter anything out.

In [5]:
n = len(kt)
mo = {'Key': 'Key', 
      'f4X43': 'Lactation', 
      'f4X45': 'DOBRIC'}
oc = [mo[k] for k in mo]
oo = list(mo)
ec = ['ID', 'CDAT', 'BDAT', 'SID', 'DID',
      'fg1', 'VWP', 'Repro_Label', 'Repro_Date_i', 'Repro_Date_f', 'Interval_Days']
c = oc + ec

fn = dc.init.SavePath(extension='flat.txt', dirct='data', mode=0)
fg17 = 'n'*17
fg8 = 'n'*8

c4 = [c for c in fdf.columns if 'f4' in c]
c5 = [c for c in fdf.columns if 'f5' in c]

with open(fn, mode="w") as f:
    f.write('\t'.join(c) + '\n')
    for _, row in fdf.iterrows():
        bd4 = row['f4X15']
        bd5 = row['f5X15']
        k0 = row['Key']
        rs = []
        
        b1 = bd4 == fg8
        b2 = bd5 == fg8
        b3 = bd4 == bd5
        
        if not b1 and not b2 and not b3:
            r4 = row.copy()
            r4['Key'] = k0 + bd4
            r4.update({c: 'n'*len(row[c]) for c in c5})
            rs.append(r4)
            r5 = row.copy()
            r5['Key'] = k0 + bd5
            r5.update({c: 'n'*len(row[c]) for c in c4})
            rs.append(r5)
            
        elif b1 and not b2:
            r = row.copy()
            r['Key'] = k0 + bd5
            rs.append(r)
            
        elif not b1 and b2:
            r = row.copy()
            r['Key'] = k0 + bd4
            rs.append(r)
            
        elif b3:
            r = row.copy()
            r['Key'] = k0 + bd4
            rs.append(r)
            
        else:
            print('\n----------\nbd4 = ', bd4, '\nbd5 = ', bd5, '\n----------\n')
            print('Please check assertion error! Buggy logic!')
            assert False
            
        for r in rs:
            vs = [str(r[j]) for j in oo]
            ID = r['Key'][:17]
            CDAT = r['Key'][17:25]
            BDAT = r['Key'][25:]
            SID = ID_verifier(row=r, id_type='SID', fg=fg17)
            DID = ID_verifier(row=r, id_type='DID', fg=fg17)
            vs += [ID, CDAT, BDAT, SID, DID]
            ki, li = 0, 1
            kf, lf = 1, 2
            vwp = 'nnn'
            bk = kf <= n-1
            while bk:
                ti, tf = r[kt[ki]], r[kt[kf]]
                di, df = r[kd[ki]], r[kd[kf]]
                vti, vtf = ti in rt, tf in rt
                vdc, vdi, vdf = check_YYYYMMDD(CDAT), check_YYYYMMDD(di), check_YYYYMMDD(df)
                
                if ti == 'n':
                    l = 'nnnn'
                    ad = ['0', vwp, l, 'nnnnnnnn', 'nnnnnnnn', 'n']
                    f.write('\t'.join(vs + ad) + '\n')
                    break
                
                else:
                    if not vti:
                        ki, kf = kf, kf + 1
                    elif vti and not vdi:
                        li, lf = lf, lf + 1
                        ki, kf = kf, kf + 1
                    elif not vtf:
                        kf += 1
                    elif vtf and not vdf:
                        lf += 1
                        kf += 1
                    elif vdi and vdf:
                        did = datetime.strptime(di, "%Y%m%d")
                        dfd = datetime.strptime(df, "%Y%m%d")
                        d = (dfd - did).days
                        if vdc and vwp == 'nnn':
                            dcd = datetime.strptime(CDAT, "%Y%m%d")
                            vwp = (did - dcd).days
                        l = str(li) + ti + str(lf) + tf
                        ad = ['1', str(vwp), l, di, df, str(d)]
                        f.write('\t'.join(vs + ad) + '\n')
                        li, lf = lf, lf + 1
                        ki, kf = kf, kf + 1
                    else:
                        print("Why? Please check!")
                        print(r[kt])
                        print(r[kd])
                        assert False
                        
                    bk = kf <= n-1
                    if not bk:
                        if vti:
                            if not vdi or not vdc:
                                di = 'nnnnnnnn'
                            elif vdi and vdc and vwp == 'nnn':
                                dcd = datetime.strptime(CDAT, "%Y%m%d")
                                did = datetime.strptime(di, "%Y%m%d")
                                vwp = (did - dcd).days
                            elif vwp != 'nnn':
                                vwp = vwp
                            else:
                                print('CRF! check')
                                print(di, vwp, vdi, vdc, CDAT)
                                assert False
                            l = str(li) + 'END'
                        else:
                            di = 'nnnnnnnn'
                            l = '0END'
                        ad = ['2', str(vwp), l, di, 'nnnnnnnn', 'n'] # Potentially replace 'n' by gestation length if possible.
                        f.write('\t'.join(vs + ad) + '\n')    

NameError: name 'kt' is not defined

In [55]:
check = fdf[fdf['Key'].str.startswith('JE840003210389761')]
with open(fn[:-4] + '_check.html', 'w', encoding='utf-8') as f:
    f.write(check.to_html(index=False, border=1))

# Visualize stuff. 
From there, add more elements, let the dataset tell a story!

In [18]:
df = pd.read_csv(fn, sep='\t', nrows=300)
with open(fn[:-3] + 'html', 'w', encoding='utf-8') as f:
    f.write(df.to_html(index=False, border=1))

# Make sure it is behavng as expected

In [22]:
df = df = pd.read_csv(fn, sep='\t')
df[df['Key'].str.slice(0, 25) == 'JEUSA00006708580020120727']

,Key,Lactation,DOBRIC,ID,CDAT,BDAT,SID,DID,Repro_Label,Repro_Date_i,Repro_Date_f,Interval_Days
11110,JEUSA0000670858002012072720090806,nn,nnnnnnnn,JEUSA000067085800,20120727,20090806,JEUSA000111950696,JEUSA000067068345,1A2A,20121011,20121026,15
11111,JEUSA0000670858002012072720090806,nn,nnnnnnnn,JEUSA000067085800,20120727,20090806,JEUSA000111950696,JEUSA000067068345,2A3A,20121026,20121118,23


In [90]:
f4s = []
f5s = []
for i in range(12, 28):
    f4s.append('f4X' + str(i))
    f5s.append('f5X' + str(i))

fdf[['Key'] + f4s].head()

,Key,f4X12,f4X13,f4X14,f4X15,f4X16,f4X17,f4X18,f4X19,f4X20,f4X21,f4X22,f4X23,f4X24,f4X25,f4X26,f4X27
0,JE 20120216,,,,20091211,D,20120824,L,0,2,1,,000000,0,4,01,093
1,JE 20161013,,,,20131227,D,20180630,L,0,2,1,,000000,0,4,01,093
2,JE 20210726,,,,20170407,D,20220308,L,1,2,1,,000000,1,4,01,093
3,JE84000006726960620160505,JE,982,000370899693,20140613,D,20180619,L,0,2,1,,000000,0,4,01,093
4,JE84000006754087220151208,JE,982,000209864423,20130227,D,20161018,L,0,2,1,02,000000,0,4,01,093


In [91]:
fdf[['Key'] + f5s].head()

,Key,f5X12,f5X13,f5X14,f5X15,f5X16,f5X17,f5X18,f5X19,f5X20,f5X21,f5X22,f5X23,f5X24,f5X25,f5X26,f5X27
0,JE 20120216,,,,20100228,D,20160519,B,0,1,1,,000000,0,4,01,193
1,JE 20161013,,,,20120226,D,20200506,B,0,1,1,,000000,0,4,01,093
2,JE 20210726,,,,20160626,D,20220519,B,0,1,1,,000000,0,4,01,082
3,JE84000006726960620160505,JE,982,000370899693,20140613,D,20161202,B,0,1,1,,000000,0,4,01,093
4,JE84000006754087220151208,JE,982,000209864423,20130227,D,20160331,B,0,1,1,02,000000,0,4,01,093


In [92]:
fdf['f4X25'].value_counts()

f4X25
4    21733
6      778
5      607
7      222
8       12
0        1
Name: count, dtype: int64

In [93]:
fdf['f4X25'].value_counts(normalize=True)

f4X25
4    0.930630
6    0.033315
5    0.025992
7    0.009506
8    0.000514
0    0.000043
Name: proportion, dtype: float64

In [94]:
fdf['f5X25'].value_counts()

f5X25
4    17565
n     4500
6      627
5      468
7      182
8       10
0        1
Name: count, dtype: int64

In [95]:
fdf['f5X25'].value_counts(normalize=True)

f5X25
4    0.752152
n    0.192695
6    0.026849
5    0.020040
7    0.007793
8    0.000428
0    0.000043
Name: proportion, dtype: float64